## GRU-attention model for landmark-eye-data

Needs output data from <b>01 processing videos</b>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GRU, Attention, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as kr
import pickle
from data_split import train_valid_test_split

In [33]:
####
#parameters
test_rate = 0.15
valid_rate = 0.15
window = 30 #frame window
seed = 123456

####
#load focus videos
of = open('landmark_eye_data.obj', 'rb')
data, labels = pickle.load(of)
of.close()

####
#transform classes
from sklearn.preprocessing import OrdinalEncoder
label_nums = OrdinalEncoder().fit_transform(labels)

####
#split into train/valid/test
X_train,X_valid,X_test,y_train,y_valid,y_test = train_valid_test_split(data,label_nums,test_rate,valid_rate,window,seed)

### Model building

In [37]:
####
#parameters
gru_size = 140
out_dense = [140]
dropout = False
droprate = 0.4

####
#functional codes
face_in = tf.keras.Input(shape=(window,X_train.shape[-1]))
# RNN Block
x = face_in
x = GRU(gru_size, return_sequences=True)(x)
x = BatchNormalization()(x)
if dropout:
    x = Dropout(droprate)(x)
# Self-Attention Block
x = Attention()([x, x])
x = tf.keras.layers.GlobalMaxPooling1D()(x)
# Output Block
for d in out_dense:
    x = Dense(d, activation="relu")(x)
    if dropout:
        x = Dropout(droprate)(x)
output = Dense(units=1, activation='softmax')(x)
# Build and Compile model
model = Model(inputs=[face_in], outputs=[output])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30, 140)]    0                                            
__________________________________________________________________________________________________
gru_2 (GRU)                     (None, 30, 140)      118020      input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 30, 140)      560         gru_2[0][0]                      
__________________________________________________________________________________________________
attention_2 (Attention)         (None, 30, 140)      0           batch_normalization_2[0][0]      
                                                                 batch_normalization_2[0][0]

In [43]:
opt = SGD(lr=1e-5, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=[X_valid, y_valid], batch_size=512, epochs=1000)

Train on 9051 samples, validate on 1912 samples
Epoch 1/1000
9051/9051 [==============================] - 3s 286us/sample - loss: 0.1147 - acc: 0.9617 - val_loss: 0.3651 - val_acc: 0.8421
Epoch 2/1000
9051/9051 [==============================] - 2s 201us/sample - loss: 0.1149 - acc: 0.9606 - val_loss: 0.4870 - val_acc: 0.7877
Epoch 3/1000
9051/9051 [==============================] - 2s 195us/sample - loss: 0.1166 - acc: 0.9611 - val_loss: 0.4985 - val_acc: 0.8075
Epoch 4/1000
9051/9051 [==============================] - 2s 199us/sample - loss: 0.1128 - acc: 0.9635 - val_loss: 0.3622 - val_acc: 0.8421
Epoch 5/1000
9051/9051 [==============================] - 2s 198us/sample - loss: 0.1137 - acc: 0.9619 - val_loss: 0.3593 - val_acc: 0.8567
Epoch 6/1000
9051/9051 [==============================] - 2s 194us/sample - loss: 0.1128 - acc: 0.9638 - val_loss: 0.3551 - val_acc: 0.8567
Epoch 7/1000
9051/9051 [==============================] - 2s 198us/sample - loss: 0.1135 - acc: 0.9627 - val_los

Epoch 59/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1132 - acc: 0.9619 - val_loss: 0.3497 - val_acc: 0.8556
Epoch 60/1000
9051/9051 [==============================] - 2s 195us/sample - loss: 0.1121 - acc: 0.9635 - val_loss: 0.3660 - val_acc: 0.8593
Epoch 61/1000
9051/9051 [==============================] - 2s 201us/sample - loss: 0.1122 - acc: 0.9622 - val_loss: 0.3693 - val_acc: 0.8588
Epoch 62/1000
9051/9051 [==============================] - 2s 191us/sample - loss: 0.1125 - acc: 0.9621 - val_loss: 0.3676 - val_acc: 0.8593
Epoch 63/1000
9051/9051 [==============================] - 2s 200us/sample - loss: 0.1119 - acc: 0.9640 - val_loss: 0.3493 - val_acc: 0.8604
Epoch 64/1000
9051/9051 [==============================] - 2s 193us/sample - loss: 0.1117 - acc: 0.9625 - val_loss: 0.3472 - val_acc: 0.8604
Epoch 65/1000
9051/9051 [==============================] - 2s 198us/sample - loss: 0.1130 - acc: 0.9637 - val_loss: 0.3485 - val_acc: 0.8598
Epoch 66/1000

Epoch 117/1000
9051/9051 [==============================] - 2s 195us/sample - loss: 0.1107 - acc: 0.9641 - val_loss: 0.3472 - val_acc: 0.8598
Epoch 118/1000
9051/9051 [==============================] - 2s 205us/sample - loss: 0.1134 - acc: 0.9624 - val_loss: 0.3632 - val_acc: 0.8593
Epoch 119/1000
9051/9051 [==============================] - 2s 198us/sample - loss: 0.1110 - acc: 0.9627 - val_loss: 0.3480 - val_acc: 0.8609
Epoch 120/1000
9051/9051 [==============================] - 2s 202us/sample - loss: 0.1119 - acc: 0.9639 - val_loss: 0.3575 - val_acc: 0.8593
Epoch 121/1000
9051/9051 [==============================] - 2s 197us/sample - loss: 0.1114 - acc: 0.9632 - val_loss: 0.3853 - val_acc: 0.8567
Epoch 122/1000
9051/9051 [==============================] - 2s 198us/sample - loss: 0.1117 - acc: 0.9634 - val_loss: 0.3490 - val_acc: 0.8614
Epoch 123/1000
9051/9051 [==============================] - 2s 192us/sample - loss: 0.1109 - acc: 0.9633 - val_loss: 0.3879 - val_acc: 0.8536
Epoch 

Epoch 175/1000
9051/9051 [==============================] - 2s 204us/sample - loss: 0.1104 - acc: 0.9650 - val_loss: 0.3422 - val_acc: 0.8588
Epoch 176/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1122 - acc: 0.9624 - val_loss: 0.4028 - val_acc: 0.8468
Epoch 177/1000
9051/9051 [==============================] - 2s 190us/sample - loss: 0.1101 - acc: 0.9652 - val_loss: 0.3422 - val_acc: 0.8604
Epoch 178/1000
9051/9051 [==============================] - 2s 202us/sample - loss: 0.1114 - acc: 0.9624 - val_loss: 0.3439 - val_acc: 0.8619
Epoch 179/1000
9051/9051 [==============================] - 2s 200us/sample - loss: 0.1125 - acc: 0.9634 - val_loss: 0.3418 - val_acc: 0.8609
Epoch 180/1000
9051/9051 [==============================] - 2s 192us/sample - loss: 0.1124 - acc: 0.9630 - val_loss: 0.9529 - val_acc: 0.6616
Epoch 181/1000
9051/9051 [==============================] - 2s 198us/sample - loss: 0.1661 - acc: 0.9388 - val_loss: 2.1866 - val_acc: 0.5471
Epoch 

Epoch 233/1000
9051/9051 [==============================] - 2s 197us/sample - loss: 0.1097 - acc: 0.9639 - val_loss: 0.3143 - val_acc: 0.8651
Epoch 234/1000
9051/9051 [==============================] - 2s 190us/sample - loss: 0.1095 - acc: 0.9641 - val_loss: 0.3387 - val_acc: 0.8682
Epoch 235/1000
9051/9051 [==============================] - 2s 199us/sample - loss: 0.1101 - acc: 0.9634 - val_loss: 0.3408 - val_acc: 0.8687
Epoch 236/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1115 - acc: 0.9627 - val_loss: 0.3351 - val_acc: 0.8630
Epoch 237/1000
9051/9051 [==============================] - 2s 200us/sample - loss: 0.1108 - acc: 0.9638 - val_loss: 0.3153 - val_acc: 0.8656
Epoch 238/1000
9051/9051 [==============================] - 2s 200us/sample - loss: 0.1101 - acc: 0.9641 - val_loss: 0.3164 - val_acc: 0.8666
Epoch 239/1000
9051/9051 [==============================] - 2s 199us/sample - loss: 0.1092 - acc: 0.9634 - val_loss: 0.3390 - val_acc: 0.8630
Epoch 

...

Epoch 755/1000
9051/9051 [==============================] - 2s 198us/sample - loss: 0.1039 - acc: 0.9654 - val_loss: 0.3358 - val_acc: 0.8703
Epoch 756/1000
9051/9051 [==============================] - 2s 189us/sample - loss: 0.1042 - acc: 0.9650 - val_loss: 0.3231 - val_acc: 0.8572
Epoch 757/1000
9051/9051 [==============================] - 2s 197us/sample - loss: 0.1041 - acc: 0.9653 - val_loss: 0.3084 - val_acc: 0.8692
Epoch 758/1000
9051/9051 [==============================] - 2s 180us/sample - loss: 0.1033 - acc: 0.9657 - val_loss: 0.3111 - val_acc: 0.8729
Epoch 759/1000
9051/9051 [==============================] - 2s 201us/sample - loss: 0.1036 - acc: 0.9646 - val_loss: 0.3189 - val_acc: 0.8719
Epoch 760/1000
9051/9051 [==============================] - 2s 195us/sample - loss: 0.1035 - acc: 0.9657 - val_loss: 0.3195 - val_acc: 0.8692
Epoch 761/1000
9051/9051 [==============================] - 2s 190us/sample - loss: 0.1042 - acc: 0.9649 - val_loss: 0.3083 - val_acc: 0.8692
Epoch 

Epoch 813/1000
9051/9051 [==============================] - 2s 202us/sample - loss: 0.1021 - acc: 0.9663 - val_loss: 0.3020 - val_acc: 0.8750
Epoch 814/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1045 - acc: 0.9669 - val_loss: 0.3045 - val_acc: 0.8760
Epoch 815/1000
9051/9051 [==============================] - 2s 195us/sample - loss: 0.1033 - acc: 0.9649 - val_loss: 0.3138 - val_acc: 0.8734
Epoch 816/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1026 - acc: 0.9664 - val_loss: 0.3175 - val_acc: 0.8719
Epoch 817/1000
9051/9051 [==============================] - 2s 195us/sample - loss: 0.1031 - acc: 0.9660 - val_loss: 0.3034 - val_acc: 0.8745
Epoch 818/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1030 - acc: 0.9656 - val_loss: 0.3040 - val_acc: 0.8740
Epoch 819/1000
9051/9051 [==============================] - 2s 190us/sample - loss: 0.1033 - acc: 0.9657 - val_loss: 0.3072 - val_acc: 0.8724
Epoch 

Epoch 871/1000
9051/9051 [==============================] - 2s 193us/sample - loss: 0.1023 - acc: 0.9661 - val_loss: 0.3260 - val_acc: 0.8724
Epoch 872/1000
9051/9051 [==============================] - 2s 199us/sample - loss: 0.1026 - acc: 0.9662 - val_loss: 0.3924 - val_acc: 0.8436
Epoch 873/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1030 - acc: 0.9645 - val_loss: 0.3066 - val_acc: 0.8745
Epoch 874/1000
9051/9051 [==============================] - 2s 194us/sample - loss: 0.1026 - acc: 0.9649 - val_loss: 0.2992 - val_acc: 0.8719
Epoch 875/1000
9051/9051 [==============================] - 2s 191us/sample - loss: 0.1026 - acc: 0.9650 - val_loss: 0.2991 - val_acc: 0.8750
Epoch 876/1000
9051/9051 [==============================] - 2s 192us/sample - loss: 0.1028 - acc: 0.9652 - val_loss: 0.3460 - val_acc: 0.8661
Epoch 877/1000
9051/9051 [==============================] - 2s 201us/sample - loss: 0.1030 - acc: 0.9660 - val_loss: 0.3434 - val_acc: 0.8687
Epoch 

Epoch 929/1000
9051/9051 [==============================] - 2s 185us/sample - loss: 0.1026 - acc: 0.9652 - val_loss: 0.3246 - val_acc: 0.8771
Epoch 930/1000
9051/9051 [==============================] - 2s 189us/sample - loss: 0.1026 - acc: 0.9655 - val_loss: 0.2980 - val_acc: 0.8808
Epoch 931/1000
9051/9051 [==============================] - 2s 197us/sample - loss: 0.1028 - acc: 0.9645 - val_loss: 0.2918 - val_acc: 0.8823
Epoch 932/1000
9051/9051 [==============================] - 2s 188us/sample - loss: 0.1024 - acc: 0.9655 - val_loss: 0.3004 - val_acc: 0.8729
Epoch 933/1000
9051/9051 [==============================] - 2s 193us/sample - loss: 0.1015 - acc: 0.9660 - val_loss: 0.2922 - val_acc: 0.8797
Epoch 934/1000
9051/9051 [==============================] - 2s 187us/sample - loss: 0.1014 - acc: 0.9656 - val_loss: 0.3003 - val_acc: 0.8792
Epoch 935/1000
9051/9051 [==============================] - 2s 197us/sample - loss: 0.1034 - acc: 0.9653 - val_loss: 0.2988 - val_acc: 0.8719
Epoch 

Epoch 987/1000
9051/9051 [==============================] - 2s 196us/sample - loss: 0.1028 - acc: 0.9655 - val_loss: 0.3157 - val_acc: 0.8781
Epoch 988/1000
9051/9051 [==============================] - 2s 191us/sample - loss: 0.1004 - acc: 0.9667 - val_loss: 0.3110 - val_acc: 0.8813
Epoch 989/1000
9051/9051 [==============================] - 2s 194us/sample - loss: 0.1004 - acc: 0.9659 - val_loss: 0.3046 - val_acc: 0.8818
Epoch 990/1000
9051/9051 [==============================] - 2s 190us/sample - loss: 0.1006 - acc: 0.9657 - val_loss: 0.2893 - val_acc: 0.8834
Epoch 991/1000
9051/9051 [==============================] - 2s 194us/sample - loss: 0.1020 - acc: 0.9656 - val_loss: 0.3025 - val_acc: 0.8813
Epoch 992/1000
9051/9051 [==============================] - 2s 190us/sample - loss: 0.1004 - acc: 0.9657 - val_loss: 0.2892 - val_acc: 0.8808
Epoch 993/1000
9051/9051 [==============================] - 2s 199us/sample - loss: 0.1020 - acc: 0.9661 - val_loss: 0.2918 - val_acc: 0.8755
Epoch 

### Testing

In [44]:
model.evaluate(X_test, y_test)

1903/1903 [==============================] - 1s 563us/sample - loss: 0.1752 - acc: 0.9275


[0.17516063830873493, 0.9274829]